In [1]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.1 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from google.colab import userdata

API_KEY = userdata.get('GOOGLE_API_KEY');
MODEL = "gemini-2.0-flash";
GEMINI_BASE_URL= "https://generativelanguage.googleapis.com/v1beta/openai/";

In [22]:
from agents import Agent, Runner, OpenAIChatCompletionsModel, AsyncOpenAI, function_tool, ItemHelpers

In [5]:
external_client = AsyncOpenAI(api_key=API_KEY, base_url=GEMINI_BASE_URL)

In [6]:
chat_completion_model = OpenAIChatCompletionsModel(openai_client=external_client, model=MODEL)

In [7]:
from agents import set_default_openai_client, set_default_openai_api, set_tracing_disabled

set_default_openai_client(client=external_client, use_for_tracing=False)
set_default_openai_api("chat_completions")
set_tracing_disabled(disabled=True)

In [25]:
import random

@function_tool
def how_many_jokes(num) -> int:
    return random.randint(1, 10)


In [58]:
import random
from openai.types.responses import ResponseTextDeltaEvent
import asyncio

@function_tool
def how_many_jokes(num: int, type = 'Software Developer') -> int:
    return random.randint(1, num)

async def main():
    agent = Agent(
        name="Joker",
        instructions=(
            "Call the `how_many_jokes` tool with a number as a parameter. "
            "Provide the number of jokes specified in the input."
        ),
        tools=[how_many_jokes],
        model=chat_completion_model
    )
    runner = Runner.run_streamed(starting_agent=agent, input="I want 5 jokes.")

    async for event in runner.stream_events():
        if event.type == "run_item_stream_event":
            if event.item.type == "tool_call_output_item":

                for tool in event.item.agent.tools:
                    print(f"-- Tool Name: {tool.name}")
                    # Extract and display all parameters of the tool
                    if hasattr(tool, 'params_json_schema') and 'properties' in tool.params_json_schema:
                        for param_name, param_details in tool.params_json_schema['properties'].items():
                            print(f"--- Parameter Name: {param_name}")
                            print(f"--- Parameter Details: {param_details}")
                    else:
                        print("--- No Parameters Found")


In [59]:
# Run the main function
asyncio.run(main())

-- Tool Name: how_many_jokes
--- Parameter Name: num
--- Parameter Details: {'title': 'Num', 'type': 'integer'}
--- Parameter Name: type
--- Parameter Details: {'default': 'Software Developer', 'title': 'Type'}
